Este código implementa um web crawler que acessa a Biblioteca Digital de Obras Raras (BDOR) da Universidade Federal do Rio de Janeiro (UFRJ), que utiliza a plataforma DSpace para gerenciar seus recursos. O script foi desenvolvido em Python e utiliza as bibliotecas requests_html e requests para realizar requisições HTTP, renderizar o conteúdo das páginas HTML e baixar os arquivos PDF.
Principais Funcionalidades:

        1- Coleta de informações do documento: O código acessa as páginas dos documentos no site da biblioteca usando o ID sequencial de cada obra.
        2- Geração de nomes de arquivos: O nome do arquivo PDF é gerado a partir do autor, título e data de publicação da obra, com limite de 63 caracteres, para facilitar a organização e evitar nomes de arquivos muito longos.
        3-  Download automático de PDFs: Após acessar a página de cada documento, o script localiza o botão para download do PDF e baixa o arquivo correspondente, verificando se o download foi bem-sucedido (tamanho do arquivo maior que 0).
        4- Tratamento de erros: O código trata adequadamente erros de conexão, arquivos não encontrados e URLs inválidas, garantindo que o processo continue mesmo em caso de falhas.

Detalhes Técnicos:

    Bibliotecas: requests_html para renderização do HTML e interação com os elementos DOM, e requests para download dos PDFs.
    Sessões: O script utiliza HTMLSession para gerenciar cookies e manter uma sessão contínua ao acessar diferentes páginas do site.
    Tratamento de exceções: O código captura exceções comuns que podem ocorrer ao acessar URLs inválidas ou não existentes (erro 404) e ao tentar processar arquivos que não estão disponíveis para download.

In [1]:
from requests_html import HTMLSession  # Biblioteca para gerenciar sessões HTML e renderizar páginas
import requests  # Biblioteca para realizar requisições HTTP
import warnings  # Biblioteca para gerenciar warnings
warnings.filterwarnings("ignore")  # Ignora todos os warnings que podem surgir durante a execução
import os  # Biblioteca para interagir com o sistema de arquivos

# Definindo o User-Agent para simular um navegador comum
DEFAULT_USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'

session = HTMLSession()  # Inicializa uma sessão HTML para gerenciar requisições contínuas
headers = {'User-Agent': DEFAULT_USER_AGENT}  # Define o cabeçalho da requisição HTTP com o User-Agent

# Definindo URLs principais da biblioteca digital
DOMAIN = 'https://bdor.sibi.ufrj.br/handle/doc/'  # URL base para acessar os documentos
SHORT_DOMAIN = 'https://bdor.sibi.ufrj.br'  # URL base do domínio principal
MAX_LENGTH = 63  # Comprimento máximo permitido para o nome do arquivo (ajustado para evitar nomes muito longos)


# Função para tratar o nome do autor, retorna o primeiro e segundo nome concatenados
def parse_author(author):
    list = author.split(" ")  # Divide o nome completo do autor em partes
    if len(list) > 1:  # Verifica se há pelo menos dois nomes
        return list[0] + list[1]  # Retorna o primeiro e segundo nomes concatenados
    else:
        return author  # Caso contrário, retorna o nome completo


# Função para gerar o nome do arquivo PDF
def generate_file_name(rawAuthor, title, issued_date):
    author = parse_author(rawAuthor)  # Pega o nome tratado do autor
    maxTitleLength = MAX_LENGTH - 6 - len(author)  # Calcula o espaço restante para o título
    partialTitle = title[0:maxTitleLength].replace(":", "_")  # Pega a parte do título que cabe e substitui ":" por "_"
    file_name = author + '@' + partialTitle + '$' + str(issued_date)  # Gera o nome do arquivo com o formato "autor@título$data"
    return file_name  # Retorna o nome do arquivo


# Função para pegar a URL do PDF na página
def get_pdf_url(response):
    buttons = response.html.find('.btn-primary')  # Procura por botões de download com a classe 'btn-primary'
    if len(buttons) > 1:  # Verifica se há mais de um botão (geralmente o segundo é o PDF)
        pdf_url = SHORT_DOMAIN + buttons[1].attrs.get('href')  # Concatena a URL do botão com o domínio
        return pdf_url  # Retorna a URL do PDF
    else:
        print("Botão de download não encontrado.")  # Caso não encontre o botão, imprime um aviso
        return None


# Função para verificar o tamanho do arquivo PDF após o download
def get_file_size(file_name):
    try:
        return os.path.getsize('./ufrj/' + file_name + '.pdf')  # Verifica o tamanho do arquivo na pasta './ufrj/'
    except FileNotFoundError:  # Caso o arquivo não exista, retorna tamanho 0
        return 0


# Função para fazer o download do PDF
def download_pdf(url, file_name):
    if url:  # Se a URL do PDF for válida
        response = requests.get(url, headers=headers, verify=False)  # Faz a requisição HTTP para baixar o PDF
        # Define o caminho onde o PDF será salvo, na pasta './ufrj/' com o nome gerado
        with open('./ufrj/' + file_name + '.pdf', 'wb') as w:  # Abre um arquivo no modo 'write binary'
            w.write(response.content)  # Escreve o conteúdo do PDF no arquivo

        # Verifica se o tamanho do arquivo é 0, ou seja, se o download falhou
        if get_file_size(file_name) == 0:
            print(f"Tamanho do arquivo {file_name} é 0, tentando novamente.")  # Imprime uma mensagem de erro
            download_pdf(url, file_name)  # Tenta fazer o download novamente
    else:
        print(f"URL do PDF inválida para o arquivo {file_name}.")  # Imprime uma mensagem de erro caso a URL seja inválida


# Função principal que realiza o download de um livro pelo ID
def download_book(bookId):
    url = DOMAIN + str(bookId)  # Constrói a URL do livro a partir do ID
    response = session.get(url, headers=headers, verify=False)  # Faz uma requisição GET à página do livro
    response.html.render()  # Renderiza a página para processar o conteúdo dinâmico

    if response.status_code == 404:  # Verifica se a página não existe (erro 404)
        print(f'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ID {str(bookId)} NÃO EXISTE! XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n\n')
        return

    # Procura o autor, título e data de publicação nos elementos HTML da página
    author = response.html.find('.author')
    title = response.html.find('.metadataFieldValue')
    issued_date = response.html.find('.metadataFieldValue')

    # Se encontrou o autor, título e data, procede com o download
    if author and title and issued_date:
        file_name = generate_file_name(author[0].text, title[1].text, issued_date[9].text)  # Gera o nome do arquivo
        pdf_url = get_pdf_url(response)  # Pega a URL do PDF
        download_pdf(pdf_url, file_name)  # Faz o download do PDF
        print(f"================================== DOWNLOAD COMPLETO ID {bookId} ==================================\n\n")
    else:
        print(f"Erro ao extrair metadados do livro ID {bookId}.")  # Caso não consiga extrair os dados, imprime um aviso


# Loop para baixar livros com IDs de 1 a 10
for id in range(1, 10):  # Itera de 1 a 10 (altere o range conforme necessário, que deve ser testado na tentativa e erro até mensurar o tamanho total do acervo)
    print(f"\n==================================== BAIXANDO {str(id)} ====================================")
    try:
        download_book(id)  # Tenta baixar o livro com o ID atual
    except Exception as e:  # Captura qualquer erro que possa ocorrer
        print(f'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ERRO no ID {str(id)}: {e} XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n\n')
        pass  # Continua o loop mesmo após erros



==================================== BAIXANDO 1 ====================================
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ERRO no ID 1: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead. XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX



==================================== BAIXANDO 2 ====================================
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ERRO no ID 2: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead. XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX



==================================== BAIXANDO 3 ====================================
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ERRO no ID 3: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead. XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX



==================================== BAIXANDO 4 ====================================
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ERRO no ID 4: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead. XXXXXXXXXXXXXXXXXXXX